In [33]:
# import module

import pickle

from core.JIG import JIG
from core.SG2 import SG2 
from core.AWL import AWL

In [66]:
def main(benign_big_group_path, malware_big_group_path, benign_signature_path, malware_signature_path, 
		 window_size, K, M, # MV2 파라미터
		 thetaJ,  # JIG 파라미터
		 vector_size, eps, minpts, ngram, hh1_size, hh2_size, hh3_size, ratio # SG2, AWL 파라미터
		):
	
	with open('./datasets/pkl/pe_benign_string.pkl', 'rb') as f:
		benign_payloads = pickle.load(f)
		
	with open(benign_big_group_path, 'rb') as f:
		benign_big_group_indices = pickle.load(f)
	
	with open('./datasets/pkl/pe_string.pkl', 'rb') as f:
		malware_payloads = pickle.load(f)

	with open(malware_big_group_path, 'rb') as f:
		malware_big_group_indices = pickle.load(f)
	
	benign_big_group_payloads = []
	benign_non_big_group_paylaods = []

	for idx, payload in enumerate(benign_payloads):
		if idx in benign_big_group_indices:
			benign_big_group_payloads.append(payload)
		else:
			benign_non_big_group_paylaods.append(payload)

	malware_big_group_payloads = []
	malware_non_big_group_paylaods = []

	for idx, payload in enumerate(malware_payloads):
		if idx in malware_big_group_indices:
			malware_big_group_payloads.append(payload)
		else:
			malware_non_big_group_paylaods.append(payload)
	
	# 시그니처 생성
	print('benign')
	cluster_signatures = SG2(payloads=benign_big_group_payloads, window_size=window_size, vector_size=vector_size, 
							 eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)
	
	with open(benign_signature_path, 'wb') as f:
		pickle.dump(cluster_signatures, f)

	print('malware')
	cluster_signatures = SG2(payloads=malware_big_group_payloads, window_size=window_size, vector_size=vector_size, 
							 eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)
	
	with open(malware_signature_path, 'wb') as f:
		pickle.dump(cluster_signatures, f)
	
	# stopwords = AWL(payloads=non_big_group_paylaods, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)

	# with open(stopword_path, 'wb') as f:
	# 	pickle.dump(stopwords, f)

In [70]:
# 파라미터

benign_big_group_indices_path = f'./res/PE_benign_big_group.pkl'
benign_signature_path = f'./res/PE_benign_signature.pkl'

malware_big_group_indices_path = f'./res/PE_mal_big_group.pkl'
malware_signature_path = f'./res/PE_mal_signature.pkl'

K = 64
M = 2 ** 14
thetaJ = 0.6
window_size = 4
vector_size = 512
eps = 0.4
minpts = 5
ngram = 4
hh1_size = 5000
hh2_size = 5000
hh3_size = 3000
ratio = 0.1

In [71]:
main(benign_big_group_path=benign_big_group_indices_path, malware_big_group_path=malware_big_group_indices_path,
		benign_signature_path=benign_signature_path, malware_signature_path=malware_signature_path,
        window_size=window_size, vector_size=vector_size, K=K, M=M, thetaJ=thetaJ,
        eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)

benign
chunking


100%|██████████| 8167/8167 [00:41<00:00, 197.70it/s]


start DBSCAN
end DBSCAN
make signature


100%|██████████| 2/2 [00:29<00:00, 14.79s/it]


end signature
malware
chunking


100%|██████████| 23402/23402 [01:07<00:00, 348.06it/s]


start DBSCAN
end DBSCAN
make signature


100%|██████████| 3/3 [00:44<00:00, 14.73s/it]


end signature


In [72]:
with open('res/PE_mal_signature.pkl', 'rb') as f:
    nomi_sig = pickle.load(f)
    
with open('res/PE_benign_signature.pkl', 'rb') as f:
    stopword = pickle.load(f)

In [76]:
sig_dict = dict()

for cluster, sigs in nomi_sig.items():
    for sig in sigs:
        print(sig)
        if sig[0] not in sig_dict:
            sig_dict[sig[0]] = 0
        sig_dict[sig[0]] += sig[1]
        
print(len(sig_dict))

('Windows', 107636)
('Qwertyuio', 88400)
('Trickler', 84556)
('software\\Qwertyuio', 76869)
('Software', 65342)
('http://', 53806)
('System', 50055)
('connect', 46122)
('ReadFile', 43809)
('socket', 42282)
('Server', 42281)
('FAILED', 42276)
('WinInet', 38432)
('success', 34592)
('InternetReadFile', 30795)
('version', 30749)
('IE 5.0', 30749)
('Unknown', 30749)
('Failed', 26906)
('DELETE', 26906)
('Qwertyuio\\Qwert', 26906)
('CxSocket', 26905)
(' delete', 26903)
('download', 26902)
('is prog', 24585)
('KERNEL32', 23191)
('Deleting', 23064)
('IE 3.0', 23062)
('Software\\Microsoft\\Windows\\CurrentVersion', 23062)
('IEGator', 23062)
('PdpSetup', 23061)
('https://', 23060)
('global', 23060)
('ExitProcess', 19271)
('Security', 19222)
('instal', 19220)
('WARE\\M', 19220)
('Connected', 19219)
('SilentSetup', 19218)
('HTTP/%s', 19218)
('open file', 19218)
('Failed to', 19218)
('Netscape', 19217)
('User-Agent', 19217)
('AppRequests', 19216)
('CloseHandle', 18877)
('kernel32', 18756)
('GetProcA

In [74]:
white_list = dict()

for cluster, sigs in stopword.items():
    for sig in sigs:
        print(sig)
        if sig[0] not in white_list:
            white_list[sig[0]] = 0
        white_list[sig[0]] += sig[1]

(' failed', 220966)
('create', 122241)
('create ', 119891)
('UAVAWH', 117540)
('AUAVAWH', 103436)
('detail', 101082)
(' failed: ', 77573)
('invalid', 70528)
('A_A^A]A\\_', 70521)
('database', 68174)
('A_A^A\\', 68171)
('Failed to create ', 65824)
('sqlite_', 63465)
('String', 61787)
('. Error ', 58766)
('version', 56845)
('delete', 54313)
('Delete', 47701)
('ATAVAWH', 47015)
('trigger', 44661)
('settings', 44661)
('System', 43684)
('WATAUAVAWH', 42315)
('default', 40612)
('A]A\\_^]', 39965)
('WAVAWH', 39961)
('0123456789', 37488)
('Default', 32916)
('A_A]A\\', 32909)
('out of', 30561)
('A_A^A\\_^', 30560)
('Failed to add ', 30557)
('window', 28209)
('Exception ', 28209)
('A_A^A]A\\_^]', 25859)
('McAfee', 25856)
('Failed to set ', 25856)
(' delete', 23754)
('length', 23618)
('not supported', 23511)
('WATAVH', 23508)
('A\\_^[]', 23508)
('A^A\\_^]', 23507)
('current', 21158)
('result', 21157)
('status', 21154)
('Unknown exception', 19660)
('.rdata', 19039)
('program', 18933)
('Firefox', 1

In [78]:
white_list_ = set(white_list.keys())
cnt = 0

for key, value in sig_dict.items():
    if key not in white_list_:
        cnt += 1
        print(key)
        
print(cnt)

Windows
Qwertyuio
Trickler
software\Qwertyuio
Software
http://
connect
socket
Server
FAILED
WinInet
InternetReadFile
IE 5.0
Unknown
Failed
DELETE
Qwertyuio\Qwert
CxSocket
download
is prog
KERNEL32
Deleting
IE 3.0
Software\Microsoft\Windows\CurrentVersion
IEGator
PdpSetup
https://
global
Security
instal
WARE\M
Connected
SilentSetup
HTTP/%s
open file
Failed to
Netscape
User-Agent
AppRequests
kernel32
user32
[[[[[[
kFreeSp
buffer
SSSSSSS
OemResDll
Deleted
DELETED
EFGHIJKLMN
IE 4.0
Closing
software\Qwertyuio\Trickler
SUCCES
Success
IE 3.02
 !"#$%
"%s"... 
User-Agent:
CreateMutex
user32.dll
advapi32.dll
Interface
[[[[[[[
Parameters
00,00,
ActiveX
ZhuDong
Active 
netsvcs
ICSeqCompressFrame
FFFdFFF
%-24s %-15s 
DEFAULT
ONLINE
FindResource
software\Qwertyuio\Qwert\stat
IE 5.5
software\QwertTest
Software\CLASSES\CLSID
Uninstall
IEGator.dll
IE 4.01
Waiting
attempts
DELETED 
SUCCESS
Timeout
UPDATE
TrickleRate
remove
PreInstalledApps
Qwert/1.0
ageBoxA
shell32
This program must be run under Win32
K